In [ ]:
!pip install transformers
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
!pip install faiss-gpu
!pip install sentencepiece
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu]
!pip install langchain
!pip install wolframalpha

In [ ]:
import os
import requests
import json

from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever
from transformers import T5Tokenizer, T5ForConditionalGeneration

import numpy as np

from tqdm.notebook import tqdm

import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def text_chunker(text, chunk_size = 500):
  text = text.split(' ')
  new_texts = []
  for i in range(0,len(text),chunk_size):
    if i + chunk_size > len(text):
      temp_text = text[i:]
    else:
      temp_text = text[i:i+chunk_size]
    new_texts.append(" ".join(temp_text))
  return new_texts

In [ ]:
dataset = pd.DataFrame()
dataset['id'] = None
dataset['text'] = None
dataset['source'] = None
files = os.listdir('./docs')

i = 0
for f in tqdm(files,total=len(files)):
  if f.find('.txt') > -1:
    text = open('./docs/'+f,'r',errors='ignore').read()
    text = text.replace('\n',' ')
    text = text.strip()
    while text.find('  ') > -1:
      text = text.replace('  ',' ')
    chunks = text_chunker(text,400) #400 tokens only, keeping space for word piece and prompt
    for chunk in chunks:
      temp_dict = {}
      temp_dict['id'] = i
      temp_dict['text'] = chunk
      temp_dict['source'] = './docs/'+f
      i += 1
      dataset = dataset.append(temp_dict,ignore_index = True)

  0%|          | 0/230 [00:00<?, ?it/s]

In [ ]:
document_store_faiss = FAISSDocumentStore(faiss_index_factory_str="Flat",return_embedding=True)

retriever_faiss = EmbeddingRetriever(document_store = document_store_faiss, embedding_model='sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.9/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
document_store_faiss.delete_all_documents()
document_store_faiss.write_documents(
                                dataset[['id', 'text', 'source']].rename(
                                                                  columns={
                                                                      'id':'id',
                                                                      'source' : 'source',
                                                                      'text':'content'
                                                                   }
                                                                 ).to_dict(orient='records'))
document_store_faiss.get_document_count()

                1. delete_all_documents() method is deprecated, please use delete_documents method
                For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/1045
                


Writing Documents:   0%|          | 0/293 [00:00<?, ?it/s]

293

In [ ]:
document_store_faiss.update_embeddings(retriever=retriever_faiss)

Updating Embedding:   0%|          | 0/293 [00:00<?, ? docs/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
def get_results(query, retriever, n_docs = 10):
  return [(item.content, item.to_dict()['meta']) for item in retriever.retrieve(query, top_k = n_docs)]

In [ ]:
def query(payload):
    data = json.dumps(payload)
    return data

In [ ]:
u_input = "How many Regional Holidays?"
input_prompt_start = "Answer the Query: "+u_input+" from the following Context. \n\n Context: "
res = get_results(u_input, retriever_faiss,1)

for r in res:
  input_text = input_prompt_start+res[0][0]
  headers = {"Authorization": "Bearer <add-key-here>",
               "Content-type":"application/json"}

  API_URL = "https://api-inference.huggingface.co/models/google/flan-ul2"

  data = query({"inputs": input_text,
                  "parameters":{"max_length":2,"wait_for_model":True}})

  response = requests.request("POST", API_URL, headers=headers, data=data)
  print("Answer: ",response.json()[0]['generated_text'])
  print("Source: ",res[0][0])
  print('Link: ',res[0][1]['source'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer:  as applicable at the Customer’s establishment
Source:  The associate will observe Public Holidays as applicable at the Customer’s establishment.
Link:  ./docs/international-relocation-policy-dec-2022_64.txt
